# GPU Training Helper

Simple notebook to run `train_max_accuracy.py` with Colab GPU acceleration.

**Setup:** Connect to Colab GPU via VS Code kernel selector before running.

## 1. Check GPU

In [ ]:
!nvidia-smi

## 2. Run Training Script

In [ ]:
%cd /home/tr4moryp/Projects/bulk_product_generator/models

# Full training (all 676K samples)
!python train_max_accuracy.py --save-dir saved/gpu_training

# Quick test (uncomment below, comment above)
# !python train_max_accuracy.py --sample-size 10000 --save-dir saved/gpu_test

## 3. View Logs (optional)

In [ ]:
!tail -50 logs/training_max_accuracy.log

## Script Options

```bash
python train_max_accuracy.py [OPTIONS]

--sample-size N       Use N samples (default: all 676K)
--save-dir DIR        Save directory (default: saved/max_accuracy)
--skip-baseline       Skip baseline, only do augmented training
--force-augmentation  Force augmented retraining even if targets met
--seed N              Random seed (default: 42)
```